Author: Rene Pickhardt

This notebook investigates several things: 

* [Done] how often and far does the max_flow / min_cut diviate from the payment pairs existing incoming and outgoing liquidity 
* [X] Given a twosplit what is the best split ratio? 
* [X] How do various discussed splitting strategies behave?

In [11]:
import json
import networkx as nx
import random

## Setup and data import

In [7]:
def import_channel_graph():
    # retrieve this by: lightning-cli listchannels > listchannels.json
    f = open("listchannels.json")
    jsn = json.load(f)
    G = nx.Graph()
    for channel in jsn["channels"]:
        src = channel["source"]
        dest = channel["destination"]
        cap = int(channel["satoshis"])
        G.add_edge(src,dest,cap=cap)
    return G

channel_graph = import_channel_graph()

In [19]:
def create_balanced_graph(channel_graph):
    funded_network = nx.DiGraph()
    for src,dest in channel_graph.edges():
        cap = channel_graph[src][dest]["cap"]
        bal = random.randint(0,cap)
        funded_network.add_edge(src,dest,cap=cap,capacity=bal)
        funded_network.add_edge(dest,src,cap=cap,capacity=cap-bal)
    return funded_network
    
funded_network = create_balanced_graph(channel_graph)

In [24]:
nodes = set(funded_network.nodes)

## Experiment 1: Max Flow vs payment pairs local view: 

We assume a receiving node will not issue an invoice larger than the amount of satoshis it can theoretically receive. Similarly we assume a sending node will not try to pay an invoice if it does not have enough funds to do so. 

We want to see how ofter the maxflow is actually constrained by these two numbers given

* the actual channelGraph with real capacities
* a uniform distribution of balances (as measured before globally)

In [25]:
def compute_inbound(funded_network,node): 
    res = 0
    for n in funded_network.neighbors(node): 
        res += funded_network[n][node]["capacity"]
    return res

def compute_outbound(funded_network,node): 
    res = 0
    for n in funded_network.neighbors(node): 
        res += funded_network[node][n]["capacity"]
    return res


In [27]:
res = []
for i in range(100):
    payment_pair = random.sample(nodes,2)
    src, dest = payment_pair
    cut_value, partition = nx.minimum_cut(funded_network, src, dest)
    reachable, non_reachable = partition
    outbound = compute_outbound(funded_network,src)
    inbound = compute_inbound(funded_network, dest)
    print(src[:10],dest[:10], cut_value, outbound, inbound)
    lim = min(outbound,inbound)
    res.append(cut_value/lim)
print(res)

03ab5de6f2 034c065b63 364321 364321 2692394
031bfe85cc 027c4afa4b 63223 7651969 63223
020e9d8a93 03829249ef 32424 32424 1908544
02990c5471 02f39686dc 192916 288903 192916
03118b8941 0328e82133 20948 20948 92159
030e3685a9 02c4748d7a 24003 414783 24003
03a6e8c14a 03a99bf1dc 248373 840327 248373
02a84dc764 03ee7c5f9e 3490132 39186290 3490132
02a789e722 022297edeb 67526 67526 38215857
03589fff9f 032bfc44f5 274127 316136 46654247
036386d314 03a065349c 59903 59903 70191
0231802089 029f68cb5b 7550 19418181 7550
0204a91bb5 02ec378c56 44698 18212796 44698
03bd7009f1 0278e3f26e 91256 91256 274567
02d558f5f7 02087384bb 45929 45929 146689
031f624a7f 02357f2de8 207368 207368 404162
037209d2cf 0353851855 84903 84903 597867
03005b000a 03f259a190 22897 176458536 22897
039c72f95b 020f8363ad 2464 147660 2464
026b221b5b 02db6ff6cf 532453 4124436 532453
03c7ef7eb3 0277b90db8 89420 89420 6926880
03a80f74c0 02f765f7dc 31470 31470 235899
03e3ae247e 02588bb45b 14440295 40093627 14440295
032c9bab7b 03681493f4

## Experiment 2: optimal splitting ratio

We want to emperically estimate the expected number of attempts needed for various splitting ratios. This is to test the hypothesis that an equidistant splitting for multipart payments works best

In [28]:
## TODO next

## Experiment 3: various splitting strategies

we want to test at least three splitting strategies

### Proposed greedy strategy: 
* compute the theoretic optimal split
* send out onions
* merge failed onions
* update knowledge of the graph
* repeat with the remaining amount

## c-lightning split
* do 10 attempts
* split in half
* repeat each half 

## Binary splitter
* not clear how to implement